# Task 5 - Train User-specific Classifiers

## 1. Introduction

<div style="text-align:justify; width: 97%">
In this task, we will complete the implementation of a full-stack P300 speller. We start by training the P300 signals classifier using stepwise linear discriminant analysis algorithm (SWLDA) in the offline portion. The classifier will classify whether the EEG signals recorded in a certain time interval come from the target stimulus, which corresponds to cases where the desired character is flashed. The assumption here is that only the target stimulus will elicit a unique waveform called P300 waves. The accuracy of the classifier plays a crucial role in the P300 speller's performance since it determines how precisely the speller can recognize the intended characters in the user's mind.
</br>

During the online portion (using the speller in a real-time scenario), we initialize the probability of each character either from a uniform distribution or a language model. To "type" each character via the P300 speller, users are asked to focus on the desired character and count the number of times it gets flashed. The screen will flash multiple characters each time, which will correspondingly trigger the brain to generate P300 waves or not. The classifier will process these signals and give the classification score. The score will be used to update the probabilities with Bayesian inference until a threshold is met or the maximum flashes are reached. The intuition here is that the P300 speller will increase the probabilities of those characters whose scores indicate that they are more likely to be in the class of targets, while at the same time, lowering the probabilities of those who are more likely to be non-targets. In the end, the character with the highest probability will be selected.
</div>

<center>

```mermaid
flowchart LR
    direction TB
    subgraph "offline portion"
    direction LR
    a1("Feed <br> training data")-->a2("Train classifier")
    a2-->a3("Get distribution of <br> classifier scores <br> for targets and non-targets")
    a3-- KDE -->a4("Derive PDF of <br> H0 and H1")
    end
    subgraph "online portion"
    direction LR
    b1("Initialize <br> characters <br> probabilities")-->b2("New flash")
    b2("New flash")-->b3("Record <br> EEG signals")
    b3("Record <br> EEG signals")-->b4("Return <br> classifier scores")
    b4("Return <br> classifier scores")-->b5("Update characters <br> probabilities")
    b5("Bayesian update <br> characters <br> probabilities")-->b6{"Decide whether <br> to stop"}
    b6-- Yes -->b7("Select character")
    b6-- No  -->b2
    end
    a4-->b5
```

</center>

<caption><center><b>Figure 1</b>: The workflow of the P300 speller development with the DS stopping rule </font></center></caption>

In [2]:
# Python standard libraries
import math

# Packages for computation and modelling
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import norm
import mne
import pickle

# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Self-defined packages
from swlda import SWLDA
import utils

# Magic command to reload packages whenever we run any later cells
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2. Set-up

<div style="text-align:justify; width: 97%">

Now, let's start by defining some "constant variables" to set up the general configuration. First, we set `board` as the keyboard interface for users to "type" characters. The keyboard is displayed on the screen during the experiment, and it's usually a $6 \times 6$ or $9 \times 8$ matrix. Here, we use a $9 \times 8$ keyboard, with 9 rows and 8 columns. The layout looks like this:

```
+---+---+---+----+
| A | B | C | .. |
+---+---+---+----+
| I | J | K | .. |
+---+---+---+----+
| Q | R | S | .. |
+---+---+---+----+
| : | : | : |    |
+---+---+---+----+
```

After setting up the board, we can define the number of rows(`n_rows`), the number of columns (`n_cols`), and the number of characters (`M`). These variables turn out to be very useful in the later analysis. Finally, we initilize the starting probability of each character equally as $1/M$. The probabilities are stored in a `np.array` with the same layout as the `board`. Note that there are some other ways to set the initial probs, but using the uniform distribution here is convenient and also intuitive.

</div>

In [3]:
BOARD = [["A",    "B",  "C",   "D",    "E",    "F",     "G",    "H"    ],
         ["I",    "J",  "K",   "L",    "M",    "N",     "O",    "P"    ],
         ["Q",    "R",  "S",   "T",    "U",    "V",     "W",    "X"    ],
         ["Y",    "Z",  "Sp",  "1",    "2",    "3",     "4",    "5"    ],
         ["6",    "7",  "8",   "9",    "0",    "Prd",   "Ret",  "Bs"   ],
         ["?",    ",",  ";",   "\\",   "/",    "+",     "-",    "Alt"  ],
         ["Ctrl", "=",  "Del", "Home", "UpAw", "End",   "PgUp", "Shft" ],
         ["Save", "'",  "F2",  "LfAw", "DnAw", "RtAw",  "PgDn", "Pause"],
         ["Caps", "F5", "Tab", "EC",   "Esc",  "email", "!",    "Sleep"]]
BOARD  = np.array(BOARD)
N_ROWS = BOARD.shape[0]  # number of rows
N_COLS = BOARD.shape[1]  # number of columns
M = N_ROWS * N_COLS      # the number of chars on the board

<div style="text-align:justify; width: 97%">

Let's define more configuration variables. `paradigm` refers to the displaying paradigm we are using. There are three commonly used paradigms - Row-Column (RC), Checkerboard (CB), and Random-paradigm (RD). I personally find this [article](https://sapienlabs.org/lab-talk/implementations-of-the-p300-bci-speller/) to be very helpful as a brief introduction to the three.
</br>

Next, we define `NUM_TIMESTAMPS` and `EPOCH_SIZE`. It's hard to explain these two without the context, so let's move back to the sampling procedure. As we know, the P300 speller tries to capture the P300 wave, a special event-related potential (ERP) elicited by the infrequent target stimuli among high-probability non-target items. As Figure 2 shows, The EEG of the P300 surfaces as a positive deflection in voltage with a latency (delay between stimulus and response) of roughly $250$ to $500$ ms (Polich, 2009). On the opposite, the EEG of normal waves doesn't have this shape. Given this, we set the length of the observation time window to be $800$ ms, starting from the release of the stimuli till $800$ ms later. In `MNE`, they introduce the data structure `Epoch` to manage the EEG data in equal-duration chunks. The official [tutorial](https://mne.tools/dev/auto_tutorials/epochs/10_epochs_overview.html) on `Epoch` should suffice to serve as a quick reference here.
</br>

Now that we have the nicely chopped data chunk (`Epoch`), let's work on sampling the electrical signals. The default sampling frequency is $256$ Hz. In other words, we take a snapshot of the EEG every $1/256$ seconds to record a voltage value. This means we will have roughly $206$ ($256 \times 0.8$) data points (or timestamps) in one `Epoch`. If we use all the timestamps to build the classifier, the model will be very heavy and hard to train. Therefore, we choose to **average** the voltage value every $13$ timestamps, which will give us $15$ averaged observations in each `Epoch`. The last $11$ timestampes are not important, so we can just drop them away. At this point, we should be clear about the meaning of the two variables, `NUM_TIMESTAMPS` and `EPOCH_SIZE`. These two specify that we need the signals of the first $195$ ($13 \times 15$) timestamps to compute $15$ observations in each `Epoch`.

</div>

<center>

<img src="./images/target_vs_nontarget_eeg.png" alt="drawing" width="500"/>

</center>

<caption><center><b>Figure 2</b>: The average EEG of target response and non-target response (8 core electrodes) </font></center></caption>

<div style="text-align:justify; width: 97%">

The original `.edf` file contains the EEG signals collected from a bunch of electrodes, but we only need the 8 core electrodes according to Krusienski et al (2006). `CORE_CHANNELS` specifies the name of these core electrodes. Note that these electrodes are called "channels" in `MNE`, and to get their exact names, we need to read/load the `.edf` data as a `Raw` object first, and then retrieve the attribute of `info` to get a brief summary of the raw data, including the channels information.

Finally, to scale up the data analysis process, we organize the data in a systematic naming convention. Each experiment object has a training session and a testing session. In each session, each one uses the P300 speller to type some words. `NUM_TRAIN_WORDS` and `NUM_TEST_WORDS` define the number of words to type during training and testing respectively. We have 13 participants in total, and they are indexed by the variable `obj` here. Keep in mind that `obj` is not consecutive, so we need to explicitly store these indices in a list when we process all participants together later. But now, let's just focus on Participant #01, and see his/her performance.

</div>

In [4]:
paradigm       = 'RC'  # display paradigm ('RC', 'CB', or 'RD')
NUM_TIMESTAMPS = 195   # number of timestamps in each window to record signals
EPOCH_SIZE     = 15    # required number of features in every epoch
CORE_CHANNELS  = ('EEG_Fz', 'EEG_Cz',  'EEG_P3',  'EEG_Pz',
                  'EEG_P4', 'EEG_PO7', 'EEG_PO8', 'EEG_Oz')
NUM_CORE_CHANNELS  = len(CORE_CHANNELS)  # number of core eletrodes
NUM_TRAIN_WORDS = 5 # number of training words for one participant
NUM_TEST_WORDS  = 5 # number of testing words for one participant

obj = 1 # the index of experiment object (participant)
obj = str(obj) if obj > 10 else '0'+str(obj)
directory = '/Users/zionshane/Desktop/Duke/Research/BCI_data/EDFData-StudyA'
obj_directory = directory + f'/A{obj}/SE001'

<div style="text-align:justify; width: 97%">

Starting from reading the input `.edf` data files, there are several steps ahead before finishing the preparation of the training/testing datasets. To structurize the code here, We wrap everything up into a function called `load_data`. Here is a short summary of what it does.

First, complete the path to the target file by setting the session (training/testing), the participant, and the paradigm given by the parameters. Then, loop through all the words in the directory (each word corresponds to a file). For each one, we:
1. Read and load the input `.edf` file, and filter out the noise signals of a certain frequency.
2. Select the core channels. Then, Build the epochs by dividing the data into many $800$ ms chunks, each of which starts from the beginning of each simulation. Tag these epochs with "targets" or "non-targets". Add all these epochs to a giant list.

Now, after we collect all the epochs, we loop through the list, and for each one, we:
1. Downsample (average) the data so each epoch now has 15 observations for each core channel. Then, concatenate these 8 core channels together to get $120 \ (=8 \times 15)$ features.
2. Separate the data into features and responses. Collect all features together, and all responses together. Return the two as the `all_features` (`np.array` with size of (`num_records`, `num_features`)) and `all_response` (`np.array` with size of (`num_records`, 1)).

</div>

In [ ]:
train_features,train_response = utils.load_data(dir=obj_directory,
                                                obj=obj,
                                                num_timestamps=NUM_TIMESTAMPS,
                                                epoch_size=EPOCH_SIZE,
                                                num_channels=NUM_CORE_CHANNELS,
                                                type=paradigm,
                                                mode='train',
                                                num_words=NUM_TRAIN_WORDS)

In [ ]:
test_features,test_response   = utils.load_data(dir=obj_directory,
                                                obj=obj,
                                                num_timestamps=NUM_TIMESTAMPS,
                                                epoch_size=EPOCH_SIZE,
                                                num_channels=NUM_CORE_CHANNELS,
                                                type=paradigm,
                                                mode='test',
                                                num_words=NUM_TEST_WORDS)

In [ ]:
clf = SWLDA(penter=0.1, premove=0.15)
clf.fit(train_features, train_response)
auc = clf.test(test_features, test_response)
print(f'AUC of the classifier for participant #{obj} is {auc:.3f}')

AUC of the classifier for participant #01 is 0.851


In [ ]:
# save the classifier as a standalone model file
with open(f'./model/A{obj}-model.pkl','wb') as f:
    pickle.dump(clf,f)

In [ ]:
scores = pd.DataFrame(clf.test(train_features), columns=['score'])
scores['is_target'] = train_response.astype('int')
mu_1, std_1 = norm.fit(data=scores.loc[scores['is_target'] == 1]['score'])
mu_0, std_0 = norm.fit(data=scores.loc[scores['is_target'] == 0]['score'])
var_1 = std_1**2
var_0 = std_0**2

In [ ]:
test_file = directory+'/A01/SE001/Test/RC/A01_SE001RC_Test06.edf'
raw_data = mne.io.read_raw_edf(test_file, preload=True, verbose=False)

stim_events = mne.find_events(raw=raw_data,
                              stim_channel='StimulusBegin',
                              verbose=False)
eeg_channels = mne.pick_channels_regexp(raw_data.info['ch_names'], 'EEG')
raw_data.notch_filter(freqs=60, picks=eeg_channels, verbose=False)
test_epochs = utils.get_core_epochs(raw_data)

In [ ]:
current_target_events = mne.find_events(raw_data, stim_channel='CurrentTarget',
                                        verbose=False)
current_target_appears = current_target_events[:,0]
current_target = current_target_events[:,2]
truth = utils.eventIDs_to_strings(BOARD, current_target)
print(f'In test #06, Participant #{obj} wants to type "{truth}".')

In test #06, Participant #01 wants to type "DRIVING".


In [ ]:
phases_events = mne.find_events(raw_data, stim_channel='PhaseInSequence',
                                verbose=False)
phases_appears = phases_events[:,0]
phases = phases_events[:,2]

In [ ]:
phases_appears

array([ 1032,  1160,  4968,  5864,  9672, 10568, 14376, 15272, 19080,
       19976, 23784, 24680, 28488, 29384, 33192])

In [ ]:
phases

array([1, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3, 2, 3])

In [ ]:
current_target_appears

array([ 1160,  5864, 10568, 15272, 19976, 24680, 29384])

In [ ]:
during_trail_phases = []
for i in range(1, len(phases), 2):
    start = phases_appears[i]
    end = phases_appears[i+1]
    during_trail_phases.append((start, end))

during_trail_phases # check results [passed!]

[(1160, 4968),
 (5864, 9672),
 (10568, 14376),
 (15272, 19080),
 (19976, 23784),
 (24680, 28488),
 (29384, 33192)]

In [ ]:
test_features, test_response = utils.split_data(test_epochs,
                                                n_channels=NUM_CORE_CHANNELS,
                                                n_times=NUM_TIMESTAMPS,
                                                n_samples=EPOCH_SIZE)

In [ ]:
test_features.shape

(833, 120)

In [ ]:
stim_begin_events = mne.find_events(raw=raw_data, stim_channel='StimulusBegin',
                                    verbose=False)
stim_begin_time   = stim_begin_events[:,0]

In [ ]:
flashing_schedule = {time:[] for time in stim_begin_time}
for i in range(N_ROWS):
    for j in range(N_COLS):
        ch = BOARD[i][j]
        ch_index = N_COLS * i + j + 1
        # Find stimulus events and target stimulus events.
        # Non-zero value in `StimulusBegin` indicates stimulus onset.
        stim_events       = mne.find_events(raw=raw_data,
                                            stim_channel='StimulusBegin',
                                            verbose=False)
        # Non-zero value in `StimulusType` if is target stimulus event.
        flashed_ch_events = mne.find_events(raw=raw_data,
                                            stim_channel=f'{ch}_{i+1}_{j+1}',
                                            verbose=False)

        # Label flashed character events.
        flashed_ch_time = np.isin(stim_events[:,0], flashed_ch_events[:,0])
        stim_events[flashed_ch_time,2]  = ch_index
        stim_events[~flashed_ch_time,2] = -1 # placeholder
        for k in range(len(stim_begin_time)):
            if stim_events[k, 2] != -1:
                flashing_schedule[stim_events[k, 0]].append(ch_index)

# flashing_schedule # check results [passed!]

In [ ]:
clf_scores = clf.test(data=test_features)

In [ ]:
NUM_SEQ = 7
T_MAX = (N_ROWS + N_COLS) * NUM_SEQ # maximum number of flashes in a trial
ACTUAL_T_MAX = int(len(stim_begin_time)/len(truth))
P_threshold = 0.9

In [ ]:
P_all = np.ones(shape=(N_ROWS, N_COLS)) * (1/M) # Initialize probs
num_flashes = 0
target_index = current_target[0]
target_loc   = ((target_index - 1) // N_COLS, (target_index - 1) %  N_COLS)
for k in range(len(stim_begin_time)):
    num_flashes += 1
    time = stim_begin_time[k]
    flashed = flashing_schedule[time]
    # Generate the classifier score
    y = clf_scores[k]
    # Update probabilities
    for i in range(N_ROWS):
        for j in range(N_COLS):
            ch_index = N_COLS * i + j + 1
            if (ch_index in flashed):
                likelihood = stats.norm.pdf(x=y, loc=mu_1, scale=std_1)
            else:
                likelihood = stats.norm.pdf(x=y, loc=mu_0, scale=std_0)
            P_all[i, j] = P_all[i, j] * likelihood
    # Normalize P_all
    P_all = P_all / P_all.sum()
    # Check if can stop
    if P_all.max() >= P_threshold:
        break
    if (time == current_target_appears[1]):
        break

print(f'It takes {num_flashes} flashes to stop.')
max_loc = np.unravel_index(P_all.argmax(), P_all.shape)
print('The estimated choice C* is "%s" with probability of %0.3f'
      % (BOARD[max_loc], P_all[max_loc]))
print('For reference, the probability of the true target "%s" is %0.3f'
      % (BOARD[target_loc], P_all[target_loc]))

It takes 56 flashes to stop.
The estimated choice C* is "D" with probability of 0.912
For reference, the probability of the true target "D" is 0.912


In [ ]:
trail_perform = {'truth':list(truth), 'select':[], 'times':[]}

for trail in range(len(during_trail_phases)):
    print(f'Trial #{trail+1}:')
    P_all = np.ones(shape=(N_ROWS, N_COLS)) * (1/M) # Initialize probs
    num_flashes = 0
    target_index = current_target[trail]
    target_loc   = ((target_index - 1) // N_COLS, (target_index - 1) %  N_COLS)
    start, end = during_trail_phases[trail]
    time = start
    k = 0

    while time <= end:
        num_flashes += 1
        flashed = flashing_schedule[time]
        # Generate the classifier score
        y = clf_scores[trail*ACTUAL_T_MAX + k]
        # Update probabilities
        for i in range(N_ROWS):
            for j in range(N_COLS):
                ch_index = N_COLS * i + j + 1
                if (ch_index in flashed):
                    likelihood = stats.norm.pdf(x=y, loc=mu_1, scale=std_1)
                else:
                    likelihood = stats.norm.pdf(x=y, loc=mu_0, scale=std_0)
                P_all[i, j] = P_all[i, j] * likelihood
        # Normalize P_all
        P_all = P_all / P_all.sum()
        # Check if can stop
        if P_all.max() >= P_threshold:
            break
        else:
            k += 1
            if trail*ACTUAL_T_MAX + k == len(stim_begin_time):
                break
            else:
                time = stim_begin_time[trail*ACTUAL_T_MAX + k]

    max_loc = np.unravel_index(P_all.argmax(), P_all.shape)
    trail_perform['select'].append(BOARD[max_loc])
    trail_perform['times'].append(num_flashes)

    print(f'It takes {num_flashes} flashes to stop.')
    print('The estimated choice C* is "%s" with probability of %0.3f'
        % (BOARD[max_loc], P_all[max_loc]))
    print('For reference, the probability of the true target "%s" is %0.3f'
        % (BOARD[target_loc], P_all[target_loc]))

trail_perform # check results [passed!]

Trial #1:
It takes 56 flashes to stop.
The estimated choice C* is "D" with probability of 0.912
For reference, the probability of the true target "D" is 0.912
Trial #2:
It takes 49 flashes to stop.
The estimated choice C* is "R" with probability of 0.959
For reference, the probability of the true target "R" is 0.959
Trial #3:
It takes 31 flashes to stop.
The estimated choice C* is "I" with probability of 0.907
For reference, the probability of the true target "I" is 0.907
Trial #4:
It takes 40 flashes to stop.
The estimated choice C* is "V" with probability of 0.973
For reference, the probability of the true target "V" is 0.973
Trial #5:
It takes 51 flashes to stop.
The estimated choice C* is "I" with probability of 0.900
For reference, the probability of the true target "I" is 0.900
Trial #6:
It takes 71 flashes to stop.
The estimated choice C* is "N" with probability of 0.999
For reference, the probability of the true target "N" is 0.999
Trial #7:
It takes 52 flashes to stop.
The est

{'truth': ['D', 'R', 'I', 'V', 'I', 'N', 'G'],
 'select': ['D', 'R', 'I', 'V', 'I', 'N', 'G'],
 'times': [56, 49, 31, 40, 51, 71, 52]}

In [ ]:
# test_epochs_list = []
# for i in range(6, 11):
#     i = str(i) if i >= 10 else '0'+str(i)
#     test_file = directory+f'/A{obj}/SE001/Test/RC/A{obj}_SE001RC_Test{i}.edf'
#     raw_test_data = mne.io.read_raw_edf(test_file, preload=True, verbose=False)
#     eeg_channels = mne.pick_channels_regexp(raw_test_data.info['ch_names'], 'EEG')
#     raw_test_data.notch_filter(freqs=60, picks=eeg_channels, verbose=False)
#     part_test_epochs = utils.get_core_epochs(raw_test_data)
#     test_epochs_list.append(part_test_epochs)

In [ ]:
# [ ] Plot AUC for each paticipates
# [ ] Plot accuracy for each paticipates (DS x 1 and SS x 1)
# [ ] Plot bits rate for each paticipates (DS)
# [ ] Plot the number of flashes per each character for each participant (DS)

## Reference
[1] Polich J. Updating P300: an integrative theory of P3a and P3b. Clin Neurophysiol. 2007;118(10):2128-2148. doi:10.1016/j.clinph.2007.04.019

[2] Krusienski, Dean J., et al. “A Comparison of Classification Techniques for the P300 Speller.” Journal of Neural Engineering, Nov. 2006, pp. 299–305, https://doi.org/10.1088/1741-2560/3/4/007.